In [ ]:
import torch
import torch_geometric
import networkx as nx
from torch_geometric.utils.convert import from_networkx, to_networkx
from torch_geometric.data import Batch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import discrete_diffusion.utils

In [ ]:
import importlib

importlib.reload(discrete_diffusion.utils)

# Batching

In [ ]:
graph_list = [torch_geometric.utils.from_networkx(nx.generators.grid_2d_graph(i, i)) for i in range(2, 5)]
for i in range(2, 5):
    graph_list[i - 2].x = torch.ones((i**2, 3))

In [ ]:
a = -torch.ones((6,))

In [ ]:
torch.sigmoid(torch.tensor([-1]))

In [ ]:
graph_list[2]

In [ ]:
graph_batch = torch_geometric.data.Batch.from_data_list(graph_list)

In [ ]:
graph = discrete_diffusion.utils.get_example_from_batch(graph_batch, 0)

In [ ]:
nx.draw(to_networkx(graph))

# edge index to adj

In [ ]:
def edge_index_to_adj(edge_index: torch.Tensor, num_nodes: int):
    adj = torch.zeros((num_nodes, num_nodes))
    adj[edge_index[0, :], edge_index[1, :]] = 1
    return adj

In [ ]:
G = nx.generators.grid_graph(dim=(4,))

In [ ]:
nx.draw(G, with_labels=True)

In [ ]:
data = from_networkx(G)

In [ ]:
data.edge_index.shape

In [ ]:
edge_index_to_adj(data.edge_index, data.num_nodes)

# Mask for block adjacency matrix

In [ ]:
data_list = [data, data, data]
# length_batches = x.ptr[1:] - x.ptr[:-1]
# mask = torch.block_diag(*[torch.ones(i, i) for i in length_batches])
batch = Batch.from_data_list(data_list)

In [ ]:
block_mat = edge_index_to_adj(batch.edge_index, batch.num_nodes).long()
block_mat

In [ ]:
mat_to_index = torch.arange(2, 14).reshape((3, 2, 2))
mat_to_index

In [ ]:
simple_mat = mat_to_index[0]
simple_mat

In [ ]:
batch.batch

In [ ]:
likel = simple_mat[:, block_mat]
likel.shape

In [ ]:
prio = mat_to_index[batch.batch, block_mat, :]
evid = mat_to_index[batch.batch, block_mat, block_mat, None]

In [ ]:
(likel * prio / evid).shape

In [ ]:
length_batches = batch.ptr[1:] - batch.ptr[:-1]
mask = torch.block_diag(*[torch.triu(torch.ones(i, i), diagonal=1) for i in length_batches]).type(torch.bool)

In [ ]:
mat[mask]

# Loss

In [ ]:
input = torch.randn(3, 2, requires_grad=True)
target = torch.rand(3, 2, requires_grad=False)
loss = F.binary_cross_entropy(torch.sigmoid(input), target, reduction="none")

In [ ]:
loss.shape

In [ ]:
540 / 90

In [1]:
import pickle as pkl